In [1]:
import numpy as np
import pandas as pd
import scipy.linalg

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

import sys
print(sys.version)
sys.path.append('../')
import os
import copy

from models.powertrain.bounded_powertrain import Bounded_powertrain
from models.kinematic.ideal_diff_drive import Ideal_diff_drive
from models.learning.blr_slip import SlipBayesianLinearRegression, FullBodySlipBayesianLinearRegression
from models.learning.blr_acceleration import AccelerationBayesianLinearRegression, FullBodyAccelerationBayesianLinearRegression
from models.kinematic.ICR_based import *
from models.kinematic.Perturbed_unicycle import *
from models.kinematic.enhanced_kinematic import *


from util.transform_algebra import *
from util.util_func import *

from sklearn import linear_model

3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0]


In [2]:
# import acceleration dataset

# dataset_path = '../data/ral2023_dataset/husky/boreal_snow/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/husky/grand_salon_tile_inflated/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/husky/grand_salon_left-deflated/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/boreal_snow/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/ga_hard_snow_a/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/ga_hard_snow_b/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/grand_salon_tile_b/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/warthog_tracks/boreal_mud/slip_dataset_all.pkl.pkl'
# dataset_path = '../data/ral2023_dataset/warthog_tracks/grand-axe_crusted-snow/slip_dataset_all.pkl'

# dataset_path = '../data/ral2023_dataset/husky/boreal_snow/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/husky/grand_salon_tile_inflated/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/husky/grand_salon_left-deflated/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/ga_hard_snow_a/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/ga_hard_snow_b/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/grand_salon_tile_b/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/boreal_snow/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/warthog_tracks/boreal_mud/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/warthog_tracks/grand-axe_crusted-snow/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/warthog_tracks/boreal_snow/acceleration_dataset.pkl'
dataset_path = '../data/ral2023_dataset/warthog_wheels/gravel_1/acceleration_dataset.pkl'

full_dataset = pd.read_pickle(dataset_path)
full_dataset_length = len(full_dataset)
train_dataset = full_dataset[:int(full_dataset_length/2)]
validate_dataset = full_dataset[int(full_dataset_length/2):]

print(train_dataset)

     init_icp_x  init_icp_y  init_icp_z  init_icp_roll  init_icp_pitch  \
0           0.0         0.0         0.0            0.0             0.0   
1           0.0         0.0         0.0            0.0             0.0   
2           0.0         0.0         0.0            0.0             0.0   
3           0.0         0.0         0.0            0.0             0.0   
4           0.0         0.0         0.0            0.0             0.0   
..          ...         ...         ...            ...             ...   
175         0.0         0.0         0.0            0.0             0.0   
176         0.0         0.0         0.0            0.0             0.0   
177         0.0         0.0         0.0            0.0             0.0   
178         0.0         0.0         0.0            0.0             0.0   
179         0.0         0.0         0.0            0.0             0.0   

     init_icp_yaw  calib_step  cmd_left_0  cmd_right_0  cmd_left_1  ...  \
0             0.0         0.0  -12.2

In [3]:
# isolate steady-state data

# extract cmd_body_vel arrays (input arrays)

idd_body_vel_x_str_list = []
idd_body_vel_y_str_list = []
idd_body_vel_yaw_str_list = []
for i in range(0, 40):
    str_idd_vel_x_i = 'idd_vel_x_' + str(i)
    str_idd_vel_y_i = 'idd_vel_y_' + str(i)
    str_idd_vel_yaw_i = 'idd_vel_yaw_' + str(i)
    idd_body_vel_x_str_list.append(str_idd_vel_x_i)
    idd_body_vel_y_str_list.append(str_idd_vel_y_i)
    idd_body_vel_yaw_str_list.append(str_idd_vel_yaw_i)
idd_body_vel_x_array = train_dataset[idd_body_vel_x_str_list].to_numpy()
idd_body_vel_y_array = train_dataset[idd_body_vel_y_str_list].to_numpy()
idd_body_vel_yaw_array = train_dataset[idd_body_vel_yaw_str_list].to_numpy()

x_train = np.column_stack((idd_body_vel_x_array.flatten(), idd_body_vel_yaw_array.flatten()))

# extract icp vels

str_icp_vel_x_list = []
str_icp_vel_y_list = []
str_icp_vel_yaw_list = []
for i in range(0, 40):
    str_icp_vel_x_i = 'icp_vel_x_' + str(i)
    str_icp_vel_y_i = 'icp_vel_y_' + str(i)
    str_icp_vel_yaw_i = 'icp_vel_yaw_' + str(i)
    str_icp_vel_x_list.append(str_icp_vel_x_i)
    str_icp_vel_y_list.append(str_icp_vel_y_i)
    str_icp_vel_yaw_list.append(str_icp_vel_yaw_i)
icp_vel_x_array = train_dataset[str_icp_vel_x_list].to_numpy()
icp_vel_y_array = train_dataset[str_icp_vel_y_list].to_numpy()
icp_vel_yaw_array = train_dataset[str_icp_vel_yaw_list].to_numpy()

# extract body_vel_distruptions arrays (output arrays)

str_body_vel_disturption_x_list = []
str_body_vel_disturption_y_list = []
str_body_vel_disturption_yaw_list = []
for i in range(0, 40):
    str_body_vel_disturption_x_i = 'body_vel_disturption_x_' + str(i)
    str_body_vel_disturption_y_i = 'body_vel_disturption_y_' + str(i)
    str_body_vel_disturption_yaw_i = 'body_vel_disturption_yaw_' + str(i)
    str_body_vel_disturption_x_list.append(str_body_vel_disturption_x_i)
    str_body_vel_disturption_y_list.append(str_body_vel_disturption_y_i)
    str_body_vel_disturption_yaw_list.append(str_body_vel_disturption_yaw_i)
    
body_vel_disturption_x_array = train_dataset[str_body_vel_disturption_x_list].to_numpy()
body_vel_disturption_y_array = train_dataset[str_body_vel_disturption_y_list].to_numpy()
body_vel_disturption_yaw_array = train_dataset[str_body_vel_disturption_yaw_list].to_numpy()

str_idd_acceleration_x_list = []
str_idd_acceleration_y_list = []
str_idd_acceleration_yaw_list = []
for i in range(0, 40):
    str_idd_acceleration_x_i = 'idd_acceleration_x_' + str(i)
    str_idd_acceleration_y_i = 'idd_acceleration_y_' + str(i)
    str_idd_acceleration_yaw_i = 'idd_acceleration_yaw_' + str(i)
    str_idd_acceleration_x_list.append(str_idd_acceleration_x_i)
    str_idd_acceleration_y_list.append(str_idd_acceleration_y_i)
    str_idd_acceleration_yaw_list.append(str_idd_acceleration_yaw_i)
idd_acceleration_x_array = train_dataset[str_idd_acceleration_x_list].to_numpy()
idd_acceleration_y_array = train_dataset[str_idd_acceleration_y_list].to_numpy()
idd_acceleration_yaw_array = train_dataset[str_idd_acceleration_yaw_list].to_numpy()

str_icp_acceleration_x_list = []
str_icp_acceleration_y_list = []
str_icp_acceleration_yaw_list = []
for i in range(0, 40):
    str_icp_acceleration_x_i = 'icp_acceleration_x_' + str(i)
    str_icp_acceleration_y_i = 'icp_acceleration_y_' + str(i)
    str_icp_acceleration_yaw_i = 'icp_acceleration_yaw_' + str(i)
    str_icp_acceleration_x_list.append(str_icp_acceleration_x_i)
    str_icp_acceleration_y_list.append(str_icp_acceleration_y_i)
    str_icp_acceleration_yaw_list.append(str_icp_acceleration_yaw_i)
icp_acceleration_x_array = train_dataset[str_icp_acceleration_x_list].to_numpy()
icp_acceleration_y_array = train_dataset[str_icp_acceleration_y_list].to_numpy()
icp_acceleration_yaw_array = train_dataset[str_icp_acceleration_yaw_list].to_numpy()

str_imu_acceleration_x_list = []
str_imu_acceleration_y_list = []
for i in range(0, 40):
    str_imu_acceleration_x_i = 'imu_acceleration_x_' + str(i)
    str_imu_acceleration_y_i = 'imu_acceleration_y_' + str(i)
    str_imu_acceleration_x_list.append(str_imu_acceleration_x_i)
    str_imu_acceleration_y_list.append(str_imu_acceleration_y_i)
imu_acceleration_x_array = train_dataset[str_imu_acceleration_x_list].to_numpy()
imu_acceleration_y_array = train_dataset[str_imu_acceleration_y_list].to_numpy()

y_train_longitudinal_slip = body_vel_disturption_x_array.flatten()
y_train_lateral_slip = body_vel_disturption_y_array.flatten()
y_train_angular_slip = body_vel_disturption_yaw_array.flatten()

# compute mean body vel disturbance for each steady-state window
n_windows = len(train_dataset)

transitory_state_mask = train_dataset['transitory_state_mask'].to_numpy() == False

transitory_state_idd_body_vel_x = idd_body_vel_x_array[transitory_state_mask]
transitory_state_idd_body_vel_y = idd_body_vel_y_array[transitory_state_mask]
transitory_state_idd_body_vel_yaw = idd_body_vel_yaw_array[transitory_state_mask]

transitory_state_icp_body_vel_x = icp_vel_x_array[transitory_state_mask]
transitory_state_icp_body_vel_y = icp_vel_y_array[transitory_state_mask]
transitory_state_icp_body_vel_yaw = icp_vel_yaw_array[transitory_state_mask]

transitory_state_body_vel_disturption_x = body_vel_disturption_x_array[transitory_state_mask]
transitory_state_body_vel_disturption_y = body_vel_disturption_y_array[transitory_state_mask]
transitory_state_body_vel_disturption_yaw = body_vel_disturption_yaw_array[transitory_state_mask]

transitory_state_idd_body_acceleration_x = icp_acceleration_x_array[transitory_state_mask]
transitory_state_idd_body_acceleration_y = icp_acceleration_y_array[transitory_state_mask]
transitory_state_idd_body_acceleration_yaw = icp_acceleration_yaw_array[transitory_state_mask]


In [4]:
# import BLR model


slip_input_dimensions = 2
a_param_init = 0
b_param_init = 0
param_variance_init = 999999999999999999999
variance_init = 1000000000

# robot = 'husky'
# robot = 'marmotte'
robot = 'warthog-track'
if robot=='husky':
    wheel_radius = 0.33 / 2
    baseline = 0.55
    dt=0.05
if robot=='marmotte':
    wheel_radius = 0.116
    baseline = 0.5927
    dt=0.05
if robot == 'warthog-track':
    wheel_radius = 0.3
    baseline = 1.1652
    dt = 0.05
kappa_param = 1

full_body_acceleration_blr = FullBodyAccelerationBayesianLinearRegression(2, 2, a_param_init, b_param_init, param_variance_init, variance_init, baseline, wheel_radius, dt, kappa_param)

In [5]:
# extract training data

# transitory_state_idd_vel_array = np.column_stack((transitory_state_idd_body_vel_x,
#                                                    transitory_state_idd_body_vel_y,
#                                                    transitory_state_idd_body_vel_yaw))
# transitory_state_icp_vel_array = np.column_stack((transitory_state_icp_body_vel_x,
#                                                   transitory_state_icp_body_vel_y,
#                                                   transitory_state_icp_body_vel_yaw))
# transitory_state_icp_acceleration_array = np.column_stack((transitory_state_idd_body_acceleration_x,
#                                                            transitory_state_idd_body_acceleration_y,
#                                                            transitory_state_idd_body_acceleration_yaw))

idd_vel_array = np.column_stack((idd_body_vel_x_array,
                                 idd_body_vel_y_array,
                                 idd_body_vel_yaw_array))
icp_vel_array = np.column_stack((icp_vel_x_array,
                                 icp_vel_x_array,
                                 icp_vel_yaw_array))
icp_acceleration_array = np.column_stack((icp_acceleration_x_array,
                                          icp_acceleration_y_array,
                                          icp_acceleration_yaw_array))

In [56]:
# train BLR models, print resulting weights

# full_body_acceleration_blr.train_params(transitory_state_idd_vel_array, transitory_state_icp_vel_array, transitory_state_icp_acceleration_array)
full_body_acceleration_blr.train_params(idd_vel_array, icp_vel_array, icp_acceleration_array)

print('weights_x : ', full_body_acceleration_blr.body_x_acceleration_blr.weights)
print('weights_yaw : ', full_body_acceleration_blr.body_yaw_acceleration_blr.weights)

# full_body_acceleration_blr.save_params('../data/ral2023_dataset/husky/boreal_snow/trained_params/acceleration/blr/')
# full_body_acceleration_blr.save_params('../data/ral2023_dataset/husky/grand_salon_tile_inflated/trained_params/acceleration/blr/')
# full_body_acceleration_blr.save_params('../data/ral2023_dataset/husky/grand_salon_left-deflated/trained_params/acceleration/blr/')
# full_body_acceleration_blr.save_params('../data/ral2023_dataset/marmotte/ga_hard_snow_a/trained_params/acceleration/blr/')
# full_body_acceleration_blr.save_params('../data/ral2023_dataset/marmotte/ga_hard_snow_b/trained_params/acceleration/blr/')
# full_body_acceleration_blr.save_params('../data/ral2023_dataset/marmotte/grand_salon_tile_b/trained_params/acceleration/blr/')
# full_body_acceleration_blr.save_params('../data/ral2023_dataset/marmotte/boreal_snow/trained_params/acceleration/blr/')
# full_body_acceleration_blr.save_params('../data/ral2023_dataset/warthog_tracks/grand-axe_crusted-snow/trained_params/acceleration/blr/')
# full_body_acceleration_blr.save_params('../data/ral2023_dataset/warthog_tracks/boreal_mud/trained_params/acceleration/blr/')
# full_body_acceleration_blr.save_params('../data/ral2023_dataset/warthog_tracks/boreal_snow/trained_params/acceleration/blr/')
full_body_acceleration_blr.save_params('../data/ral2023_dataset/warthog_wheels/gravel_1/trained_params/acceleration/blr/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/husky/grand_salon_tile_inflated/trained_params/slip/blr/steady-state/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/husky/grand_salon_left-deflated/trained_params/slip/blr/steady-state/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/marmotte/ga_hard_snow_a/trained_params/slip/blr/steady-state/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/marmotte/ga_hard_snow_b/trained_params/slip/blr/steady-state/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/marmotte/grand_salon_tile_b/trained_params/slip/blr/steady-state/')
# full_body_acceleration_blr.save_params('../data/ral2023_dataset/warthog_tracks/boreal_mud/trained_params/acceleration/blr/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/warthog_tracks/grand-axe_crusted-snow/trained_params/slip/blr/steady-state/')


weights_x :  [[-0.01143516]
 [ 0.01222352]]
weights_yaw :  [[-0.01172181]
 [ 0.01262297]]


In [13]:
# train BLR steady-state for every

steady_state_idd_body_vel_x_shuffled = copy.deepcopy(steady_state_idd_body_vel_x.flatten())
np.random.seed(12345)
np.random.shuffle(steady_state_idd_body_vel_x_shuffled)
steady_state_idd_body_vel_y_shuffled = copy.deepcopy(steady_state_idd_body_vel_y.flatten())
np.random.seed(12345)
np.random.shuffle(steady_state_idd_body_vel_y_shuffled)
steady_state_idd_body_vel_yaw_shuffled = copy.deepcopy(steady_state_idd_body_vel_yaw.flatten())
np.random.seed(12345)
np.random.shuffle(steady_state_idd_body_vel_yaw_shuffled)

steady_state_body_vel_disturption_x_shuffled = copy.deepcopy(steady_state_body_vel_disturption_x.flatten())
np.random.seed(12345)
np.random.shuffle(steady_state_body_vel_disturption_x_shuffled)
steady_state_body_vel_disturption_y_shuffled = copy.deepcopy(steady_state_body_vel_disturption_y.flatten())
np.random.seed(12345)
np.random.shuffle(steady_state_body_vel_disturption_y_shuffled)
steady_state_body_vel_disturption_yaw_shuffled = copy.deepcopy(steady_state_body_vel_disturption_yaw.flatten())
np.random.seed(12345)
np.random.shuffle(steady_state_body_vel_disturption_yaw_shuffled)

print(steady_state_idd_body_vel_x_shuffled.flatten())
print(steady_state_idd_body_vel_y_shuffled.flatten())
print(steady_state_idd_body_vel_yaw_shuffled.flatten())

# print(steady_state_body_vel_disturption_x_shuffled.flatten())

# saved_params_root_path = '../data/ral2023_dataset/husky/boreal_snow/trained_params/slip/blr/steady-state/growing_training_set/'
saved_params_root_path = '../data/ral2023_dataset/warthog_tracks/boreal_mud/trained_params/slip/blr/steady-state/growing_training_set/'
# saved_params_root_path = '../data/ral2023_dataset/marmotte/grand_salon_tile_b/trained_params/slip/blr/steady-state/growing_training_set/'

for i in range(1, steady_state_body_vel_disturption_x_shuffled.shape[0]):
    training_timesteps_path = saved_params_root_path + str(i) + '/'
    if not os.path.exists(training_timesteps_path):
        os.makedirs(training_timesteps_path)
    x_train = np.column_stack((steady_state_idd_body_vel_x_shuffled[:i], 
                               steady_state_idd_body_vel_y_shuffled[:i], 
                               steady_state_idd_body_vel_yaw_shuffled[:i]))
    y_train = np.column_stack((steady_state_body_vel_disturption_x_shuffled[:i], 
                               steady_state_body_vel_disturption_y_shuffled[:i], 
                               steady_state_body_vel_disturption_yaw_shuffled[:i]))
    
    # print(x_train)
    # print(y_train)
    full_body_slip_blr = FullBodySlipBayesianLinearRegression(1, 1, 3, a_param_init, b_param_init, param_variance_init, variance_init, baseline, wheel_radius, dt, kappa_param)
    full_body_slip_blr.train_params(x_train, y_train)
    full_body_slip_blr.save_params(training_timesteps_path)
    
print('weights_x : ', full_body_slip_blr.body_x_slip_blr.weights)
print('weights_y : ', full_body_slip_blr.body_y_slip_blr.weights)
print('weights_yaw : ', full_body_slip_blr.body_yaw_slip_blr.weights)

NameError: name 'steady_state_idd_body_vel_x' is not defined

In [ ]:
weights_x :  [[-0.4235368]]
weights_y :  [[0.12007774]]
weights_yaw :  [[-0.04434587]
 [-0.04762662]
 [ 0.71554438]]
